<a href="https://colab.research.google.com/github/monttj/computational-physics/blob/2021/ComPhy-11-ConvNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount google drive
Only if you are using colab, you need to mount your drive and go to the directory where the necessary files are located : computational physics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/computational-physics

### Convolution Neural Network

##### setup 

We will use ```Keras``` for the CNN exercise.
Keras is the Python Deep Learning library running on top of ```TensorFlow```.

In [ ]:

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import backend as K


We need following lines to use only fraction of GPU so that others can use the remaining part of GPU in Server. 

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 0.05 * 20_000)]
            )
        print(f"{len(gpus)} GPU memory limited to 5% per GPU.")
    except RuntimeError as e:
        print(e)

##### load mnist 

In [ ]:
batch_size = 32
num_classes = 10
epochs = 3

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

##### build model in keras

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

Can you check how many parameters are there to be determined? 

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Now check the loss and accuracy with test 10000 sample.

In [ ]:
model.evaluate(x_test, y_test)

##### prediction

In [ ]:
predictions = model.predict(x_test)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
img = x_test[0].reshape(28, 28)  # reshape the image to original shape 
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
y_test[0]

##### separate sample for validation 

We will use a fraction of training sample for validation. 

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

Now check overtraining using validation sample

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=3,
                    validation_data=(x_val, y_val))

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
history_dict = history.history
history_dict.keys()

##### check overfitting

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

When do you think you should stop training to avoid overtaining?

We can also check the accuracy. 

In [ ]:
plt.clf()   # clear figure
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

##### save model 

In [ ]:
model.save("tensorflow_model.h5")

##### load model

In [ ]:
from tensorflow import keras
model = keras.models.load_model("tensorflow_model.h5")